In [2]:
from imutils.video import WebcamVideoStream
from imutils.video import FPS
import argparse
import imutils
import cv2
from datetime import datetime
import mediapipe as mp
import serial
import numpy as np


objc[93196]: Class CaptureDelegate is implemented in both /Users/nicoloagostara/miniforge3/envs/hsi/lib/python3.11/site-packages/cv2/cv2.abi3.so (0x169fb25a0) and /Users/nicoloagostara/miniforge3/envs/hsi/lib/python3.11/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x1479d8860). One of the two will be used. Which one is undefined.
objc[93196]: Class CVWindow is implemented in both /Users/nicoloagostara/miniforge3/envs/hsi/lib/python3.11/site-packages/cv2/cv2.abi3.so (0x169fb25f0) and /Users/nicoloagostara/miniforge3/envs/hsi/lib/python3.11/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x1360f8a68). One of the two will be used. Which one is undefined.
objc[93196]: Class CVView is implemented in both /Users/nicoloagostara/miniforge3/envs/hsi/lib/python3.11/site-packages/cv2/cv2.abi3.so (0x169fb2618) and /Users/nicoloagostara/miniforge3/envs/hsi/lib/python3.11/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x1360f8a90). One of the two w

In [14]:
display = 2
num_frames = 2000

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(model_complexity= 1)
mp_draw = mp.solutions.drawing_utils

# Specify the serial port your Arduino is connected to
arduino_port = '/dev/cu.usbmodem21301'  # Change this to your port (e.g., 'COM3' on Windows)

# Establish serial communication
ser = serial.Serial(arduino_port, 2000000, timeout=1)

def send_coordinates(x, y):
    x = round(x, 3)
    y = round(y, 3)
    # Send x and y coordinates to Arduino
    data = f"{x} {y}\n"
    print(data)
    ser.write(data.encode())


In [15]:

# created a *threaded* video stream, allow the camera sensor to warmup,
# and start the FPS counter
print("[INFO] sampling THREADED frames from webcam...")
vs = WebcamVideoStream(src=0).start()
fps = FPS().start()
# loop over some frames...this time using the threaded stream
while fps._numFrames < num_frames:
    # grab the frame from the threaded video stream and resize it
    # to have a maximum width of 400 pixels
    frame = vs.read()
    frame = imutils.resize(frame, width=512)
    # check to see if the frame should be displayed to our screen
    frame = cv2.flip(frame, 1)
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    #img_rgb = cv2.flip(img_rgb, 0)

    results = hands.process(img_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            point = {}  
            for id, lm in enumerate(hand_landmarks.landmark):
                point[id] = (lm.x, lm.y, lm.z)
            
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            center = point[5]
            if fps._numFrames % 1 == 0:
                send_coordinates(center[0], center[1])

    if display > 0:
        if fps._numFrames > 0:
            fps_str = f"{fps._numFrames / (datetime.now() - fps._start).total_seconds():.2f}"
            #print(type(fps_str))
            cv2.putText(frame, fps_str, org = (10, 30), fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 1, color = (0, 255, 0), thickness = 1)

        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break
        # update the FPS counter
        fps.update()
    # stop the timer and display FPS information
fps.stop()
print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()

[INFO] sampling THREADED frames from webcam...
0.541 0.819

0.556 0.725

0.559 0.548

0.561 0.586

0.547 0.49

0.539 0.45

0.533 0.436

0.523 0.42

0.515 0.413

0.511 0.412

0.507 0.413

0.491 0.425

0.465 0.417

0.399 0.423

0.349 0.431

0.286 0.445

0.234 0.452

0.19 0.475

0.178 0.485

0.179 0.495

0.188 0.491

0.243 0.485

0.297 0.473

0.376 0.426

0.586 0.406

0.626 0.398

0.66 0.4

0.661 0.402

0.628 0.413

0.597 0.411

0.525 0.423

0.409 0.439

0.455 0.446

0.273 0.481

0.237 0.488

0.194 0.507

0.185 0.514

0.192 0.513

0.211 0.509

0.261 0.514

0.303 0.502

0.374 0.501

0.428 0.504

0.512 0.497

0.582 0.499

0.606 0.501

0.627 0.501

0.63 0.501

0.634 0.505

0.634 0.508

0.632 0.515

0.63 0.519

0.623 0.528

0.619 0.528

0.609 0.527

0.603 0.526

0.582 0.516

0.564 0.514

0.5 0.514

0.446 0.521

0.334 0.559

0.26 0.562

0.221 0.566

0.21 0.568

0.198 0.569

0.196 0.572

0.192 0.573

0.187 0.575

0.184 0.576

0.182 0.579

0.178 0.581

0.178 0.583

0.178 0.583

0.176 0.582

0.17

In [ ]:
0.672*180

120.96000000000001